In [1]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = r"C:\Users\varun\Downloads\archive (14)\IMDB Dataset.csv"
df = pd.read_csv(file_path)

# Display dataset information
print(df.head())
print(df.info())


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB
None


    review                                         sentiment
0   "This movie was amazing! Highly recommend it."  positive
1   "Not worth watching. It was boring."           negative


# Preprocess the Data
Convert sentiments into numerical labels and prepare the data for tokenization.

In [2]:
# Convert sentiment to numerical labels
df['label'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Split into train and test sets
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['review'], df['label'], test_size=0.2, random_state=42
)


# Tokenization
Use a Hugging Face tokenizer to process the text.

python
Copy code


In [3]:
from transformers import AutoTokenizer

# Choose a model
model_name = "distilbert-base-uncased"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)


# Create TensorFlow Datasets
Convert the tokenized data into TensorFlow datasets for model training.

In [4]:
import tensorflow as tf

# Convert to TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    list(train_labels)
)).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    list(test_labels)
)).batch(32)


# Load and Fine-Tune a Transformer Model
Fine-tune a pre-trained model on this dataset.

In [5]:
pip install --upgrade transformers tensorflow


In [18]:
import tensorflow as tf

graph = tf.compat.v1.get_default_graph()

In [19]:
from transformers import TFAutoModelForSequenceClassification, AutoConfig
import tensorflow as tf

model_name = "bert-base-uncased"  # Replace with your model name

# Load model configuration
config = AutoConfig.from_pretrained(model_name, num_labels=2)

# Load the pre-trained model
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, config=config)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

print("Model loaded and compiled successfully!")



All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded and compiled successfully!


# Evaluate the Model
Evaluate the performance on the test set.

In [20]:
results = model.evaluate(test_dataset)
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")


313/313 [==============================] - 1491s 5s/step - loss: 0.7076 - accuracy: 0.5037
Test Loss: 0.7076328992843628, Test Accuracy: 0.5037000179290771
